### Implementing Data Validation using Protobuf in a Distributed System
**Description**: Use gRPC to implement a distributed system that validates messages using
Protobuf.

**Steps**:
1. Create a .proto file for gRPC service.
2. Implement server-side validation
    - Create a gRPC server
    - Bind the server to an address
    - Start server

In [2]:
# validation.proto content (save separately as validation.proto before running this script):
'''
syntax = "proto3";

package validation;

service Validator {
  rpc ValidatePerson (Person) returns (ValidationResponse);
}

message Person {
  string name = 1;
  int32 age = 2;
  string email = 3;
}

message ValidationResponse {
  bool valid = 1;
  string message = 2;
}
'''

# --- Run this in terminal once before running the Python code below to generate gRPC files ---
# python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. validation.proto

import grpc
from concurrent import futures
import time
import threading

import validation_pb2
import validation_pb2_grpc

class ValidatorServicer(validation_pb2_grpc.ValidatorServicer):
    def ValidatePerson(self, request, context):
        if not request.name:
            return validation_pb2.ValidationResponse(valid=False, message="Name cannot be empty.")
        if request.age < 0 or request.age > 120:
            return validation_pb2.ValidationResponse(valid=False, message="Age must be between 0 and 120.")
        if "@" not in request.email:
            return validation_pb2.ValidationResponse(valid=False, message="Invalid email address.")
        return validation_pb2.ValidationResponse(valid=True, message="Validation successful.")

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    validation_pb2_grpc.add_ValidatorServicer_to_server(ValidatorServicer(), server)
    server.add_insecure_port('[::]:50051')
    server.start()
    print("Server started on port 50051...")
    try:
        while True:
            time.sleep(86400)
    except KeyboardInterrupt:
        server.stop(0)

def run_client():
    channel = grpc.insecure_channel('localhost:50051')
    stub = validation_pb2_grpc.ValidatorStub(channel)
    test_cases = [
        validation_pb2.Person(name="", age=25, email="test@example.com"),
        validation_pb2.Person(name="Bob", age=130, email="bob@example.com"),
        validation_pb2.Person(name="Carol", age=30, email="carolexample.com"),
        validation_pb2.Person(name="Dave", age=40, email="dave@example.com"),
    ]
    for person in test_cases:
        response = stub.ValidatePerson(person)
        print(f"Input: name={person.name}, age={person.age}, email={person.email}")
        print(f"Valid: {response.valid}, Message: {response.message}\n")

if __name__ == '__main__':
    # Run server in a separate thread
    server_thread = threading.Thread(target=serve, daemon=True)
    server_thread.start()
    time.sleep(2)  # wait for server to start

    # Run client requests
    run_client()

ModuleNotFoundError: No module named 'validation_pb2'